In [6]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from pycaret.regression import *
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import re

In [7]:
data=pd.read_csv(r'Z:\projects\smart water system\dataset\train.csv')

In [8]:
data

,Timestamp,Residents,Apartment_Type,Temperature,Humidity,Water_Price,Period_Consumption_Index,Income_Level,Guests,Amenities,Appliance_Usage,Water_Consumption
0,01/01/2002 00,1,Studio,15.31,46.61,1.06,0.970000,Low,0,Swimming Pool,0.0,64.85
1,01/01/2002 08,4,NaN,21.01,66.11,2.98,0.910000,Upper Middle,1,Swimming Pool,1.0,192.50
2,01/01/2002 16,2,Cottage,12.86,60.86,1.44,1.430000,Middle,0,NaN,1.0,116.62
3,02/01/2002 00,2,1BHK,20.16,50.58,1.48,0.910000,Middle,-1,Garden,0.0,76.96
4,02/01/2002 08,2,Cottage,16.23,52.25,1.14,1.110000,Middle,0,Fountain,0.0,104.70
...,...,...,...,...,...,...,...,...,...,...,...,...
13995,10/10/2014 00,2,1BHK,25.61,61.5,1.70,0.940000,Low,0,NaN,0.0,78.59
13996,10/10/2014 08,5,2BHK,13.27,52.58,1.88,1.030000,Upper Middle,0,Garden,1.0,185.50
13997,10/10/2014 16,4,2BHK,NaN,46.93,1.22,1.100000,Middle,0,NaN,1.0,180.28
13998,11/10/2014 00,4,3BHK,11.62,64.48,2.86,1.120000,Upper Middle,1,Swimming Pool,0.0,212.19


In [9]:
data.isna().sum()

Timestamp                      0
Residents                      0
Apartment_Type               426
Temperature                  441
Humidity                       0
Water_Price                    0
Period_Consumption_Index       0
Income_Level                 426
Guests                         0
Amenities                   5997
Appliance_Usage              415
Water_Consumption              0
dtype: int64

In [10]:
data['Apartment_Type'].unique()

array(['Studio', nan, 'Cottage', '1BHK', '2BHK', 'Bungalow', '3BHK',
       'Detached'], dtype=object)

In [11]:
data['Apartment_Type'] = data['Apartment_Type'].fillna('Missing')

In [12]:
data.isna().sum()

Timestamp                      0
Residents                      0
Apartment_Type                 0
Temperature                  441
Humidity                       0
Water_Price                    0
Period_Consumption_Index       0
Income_Level                 426
Guests                         0
Amenities                   5997
Appliance_Usage              415
Water_Consumption              0
dtype: int64

In [13]:
data['Amenities'].unique()

array(['Swimming Pool', nan, 'Garden', 'Fountain', 'Jacuzzi'],
      dtype=object)

In [14]:
data['Amenities'] = data['Amenities'].fillna('Missing')

In [15]:
data.isna().sum()

Timestamp                     0
Residents                     0
Apartment_Type                0
Temperature                 441
Humidity                      0
Water_Price                   0
Period_Consumption_Index      0
Income_Level                426
Guests                        0
Amenities                     0
Appliance_Usage             415
Water_Consumption             0
dtype: int64

In [16]:
data.dtypes

Timestamp                    object
Residents                     int64
Apartment_Type               object
Temperature                 float64
Humidity                     object
Water_Price                 float64
Period_Consumption_Index    float64
Income_Level                 object
Guests                        int64
Amenities                    object
Appliance_Usage             float64
Water_Consumption           float64
dtype: object

In [17]:
data['Humidity'].value_counts().sum()

14000

In [18]:
data['Humidity'].unique()

array(['46.61', '66.11', '60.86', ..., '67.75', '81.81', '64.48'],
      dtype=object)

In [19]:
valid_categories = ['Low', 'Middle', 'Upper Middle', 'Rich']

# 2. Replace unusual values with 'Unknown'
data['Income_Level'] = data['Income_Level'].apply(lambda x: x if x in valid_categories else 'Unknown')

# 3. Handle missing values (replace with 'Unknown' in this example)
data['Income_Level'] = data['Income_Level'].fillna('Missing')

In [20]:
data['Income_Level'].unique()

array(['Low', 'Upper Middle', 'Middle', 'Unknown', 'Rich'], dtype=object)

In [21]:
data['Appliance_Usage'].unique()

array([ 0.,  1., nan])

In [22]:
data['Appliance_Usage'].dtype

dtype('float64')

In [23]:
def convert_to_float_or_nan(value):
    """Converts a value to float if it's a numerical string,
    otherwise returns NaN."""
    try:
        # Check if the value matches a numerical pattern
        if re.fullmatch(r'\d+(\.\d+)?', value):
            return float(value)
        else:
            return np.nan  # Non-numerical, replace with NaN
    except (ValueError, TypeError):
        return np.nan  # Handle errors (e.g., if value is None)

# Apply the function to the 'Humidity' column
data['Humidity'] = data['Humidity'].apply(convert_to_float_or_nan)

In [24]:
data["Humidity"].isna().sum()

397

In [25]:


# Create a LabelEncoder object
encoder = LabelEncoder()

# Select the categorical columns you want to encode
categorical_columns = ['Apartment_Type', 'Amenities']  # Replace with your column names

# Loop through the columns and apply label encoding
for column in categorical_columns:
    data[column] = encoder.fit_transform(data[column])

In [26]:
data['Income_Level']=encoder.fit_transform(data['Income_Level'])

In [27]:
data=data.drop('Timestamp',axis=1)

In [28]:
data.columns

Index(['Residents', 'Apartment_Type', 'Temperature', 'Humidity', 'Water_Price',
       'Period_Consumption_Index', 'Income_Level', 'Guests', 'Amenities',
       'Appliance_Usage', 'Water_Consumption'],
      dtype='object')

In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
X=data.drop('Water_Consumption',axis=1)
y=data['Water_Consumption']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
regressor = RandomForestRegressor(n_estimators=100, random_state=42)  # Adjust parameters as needed
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 381.2471861916464


In [31]:
# Print the first 10 actual and predicted values
print("Actual:", y_test[:10])
print("Predicted:", y_pred[:10])

# Create a DataFrame to compare
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results_df.head()) # Print the first 5 rows

Actual: 2900      98.87
3143      99.29
8231      90.12
3855      88.70
8045      89.58
7653     128.64
8610     269.14
7310     217.28
10175     97.45
5538     150.11
Name: Water_Consumption, dtype: float64
Predicted: [101.5354 108.2323  97.992   91.917  107.4448 124.206  307.9555 188.6793
  99.6785 155.4515]
      Actual  Predicted
2900   98.87   101.5354
3143   99.29   108.2323
8231   90.12    97.9920
3855   88.70    91.9170
8045   89.58   107.4448


In [32]:
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
score = max(0, 100 - rmse)

print("Score:", score)

Score: 80.47444786461477


In [33]:
data.isna().sum()

Residents                     0
Apartment_Type                0
Temperature                 441
Humidity                    397
Water_Price                   0
Period_Consumption_Index      0
Income_Level                  0
Guests                        0
Amenities                     0
Appliance_Usage             415
Water_Consumption             0
dtype: int64

In [34]:
reg_setup = setup(data=pd.concat([X_train, y_train], axis=1),
                  target='Water_Consumption',
                  remove_multicollinearity=True,
                  train_size=0.8,
                  normalize=True,
                  verbose=True,
                  session_id=42)

best_model = compare_models(sort='RMSE')
print(f"Best Model: {best_model}")

,Description,Value
0,Session id,42
1,Target,Water_Consumption
2,Target type,Regression
3,Original data shape,"(11200, 11)"
4,Transformed data shape,"(11200, 11)"
5,Transformed train set shape,"(8960, 11)"
6,Transformed test set shape,"(2240, 11)"
7,Numeric features,10
8,Rows with missing values,8.6%
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,7.3365,155.4341,12.4269,0.9703,0.0665,0.0440,0.1600
gbr,Gradient Boosting Regressor,10.5826,264.0079,16.2154,0.9496,0.0887,0.0648,0.1540
et,Extra Trees Regressor,12.0820,349.0559,18.6511,0.9333,0.1005,0.0731,0.2940
rf,Random Forest Regressor,13.5740,415.5807,20.3562,0.9206,0.1074,0.0810,0.5080
dt,Decision Tree Regressor,21.1609,995.0465,31.5160,0.8096,0.1669,0.1254,0.0180
knn,K Neighbors Regressor,26.0569,1374.9468,37.0331,0.7374,0.2067,0.1635,0.0360
ada,AdaBoost Regressor,33.2732,1489.1309,38.5794,0.7142,0.2845,0.2747,0.0880
omp,Orthogonal Matching Pursuit,48.0957,3992.1381,63.1366,0.2371,0.3714,0.3352,0.0090
dummy,Dummy Regressor,57.6332,5239.9009,72.3488,-0.0018,0.4500,0.4338,0.0100
en,Elastic Net,50.2040,402426.6157,250.6135,-72.6643,0.3426,0.3324,0.0100


Best Model: LGBMRegressor(n_jobs=-1, random_state=42)


In [35]:
final_model = finalize_model(best_model)
predictions = predict_model(final_model, data=X_test)


print(f"X_test shape: {X_test.shape}")
print(f"Predictions shape: {predictions.shape}")


y_pred = predictions.iloc[:len(X_test)]['prediction_label']


rmse_score = np.sqrt(mean_squared_error(y_test, y_pred))
custom_score = max(0, 100 - rmse_score)

print(f"Final RMSE: {rmse_score:.2f}")
print(f"Custom Score: {custom_score:.2f}")


X_test shape: (2800, 10)
Predictions shape: (2800, 11)
Final RMSE: 11.78
Custom Score: 88.22


In [36]:
data.isna().sum()

Residents                     0
Apartment_Type                0
Temperature                 441
Humidity                    397
Water_Price                   0
Period_Consumption_Index      0
Income_Level                  0
Guests                        0
Amenities                     0
Appliance_Usage             415
Water_Consumption             0
dtype: int64

In [37]:
dt=data.copy()

In [38]:
dt['Temperature']=data['Temperature'].fillna(data['Temperature'].median())
dt['Humidity']=data['Humidity'].fillna(data['Humidity'].median())
dt['Appliance_Usage']=data['Appliance_Usage'].fillna(data['Appliance_Usage'].median())

In [39]:
dt.isna().sum()

Residents                   0
Apartment_Type              0
Temperature                 0
Humidity                    0
Water_Price                 0
Period_Consumption_Index    0
Income_Level                0
Guests                      0
Amenities                   0
Appliance_Usage             0
Water_Consumption           0
dtype: int64

In [40]:
data.isna().sum()

Residents                     0
Apartment_Type                0
Temperature                 441
Humidity                    397
Water_Price                   0
Period_Consumption_Index      0
Income_Level                  0
Guests                        0
Amenities                     0
Appliance_Usage             415
Water_Consumption             0
dtype: int64

In [41]:
X=dt.drop('Water_Consumption',axis=1)
y=dt['Water_Consumption']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


reg_setup = setup(data=pd.concat([X_train, y_train], axis=1),
                  target='Water_Consumption',
                  remove_multicollinearity=True,
                  train_size=0.8,
                  normalize=True,
                  verbose=True,
                  session_id=42)

best_model = compare_models(sort='RMSE')
print(f"Best Model: {best_model}")

,Description,Value
0,Session id,42
1,Target,Water_Consumption
2,Target type,Regression
3,Original data shape,"(11200, 11)"
4,Transformed data shape,"(11200, 11)"
5,Transformed train set shape,"(8960, 11)"
6,Transformed test set shape,"(2240, 11)"
7,Numeric features,10
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,7.2493,150.6505,12.2359,0.9712,0.0657,0.0436,0.1520
gbr,Gradient Boosting Regressor,10.5004,262.1792,16.1541,0.9500,0.0885,0.0643,0.1610
et,Extra Trees Regressor,11.9697,347.4899,18.6034,0.9337,0.0997,0.0722,0.2650
rf,Random Forest Regressor,13.5873,417.0273,20.3901,0.9203,0.1074,0.0810,0.4720
dt,Decision Tree Regressor,21.4878,1046.8666,32.3410,0.7996,0.1701,0.1271,0.0180
knn,K Neighbors Regressor,26.0479,1375.5635,37.0427,0.7373,0.2067,0.1634,0.0490
ada,AdaBoost Regressor,33.3465,1492.5815,38.6199,0.7135,0.2835,0.2743,0.0810
omp,Orthogonal Matching Pursuit,48.0957,3992.1381,63.1366,0.2371,0.3714,0.3352,0.0110
dummy,Dummy Regressor,57.6332,5239.9009,72.3488,-0.0018,0.4500,0.4338,0.0100
en,Elastic Net,50.2002,401974.0122,250.4959,-72.5815,0.3426,0.3324,0.0120


Best Model: LGBMRegressor(n_jobs=-1, random_state=42)


In [42]:
final_model = finalize_model(best_model)
predictions = predict_model(final_model, data=X_test)


print(f"X_test shape: {X_test.shape}")
print(f"Predictions shape: {predictions.shape}")


y_pred = predictions.iloc[:len(X_test)]['prediction_label']


rmse_score = np.sqrt(mean_squared_error(y_test, y_pred))
custom_score = max(0, 100 - rmse_score)

print(f"Final RMSE: {rmse_score:.2f}")
print(f"Custom Score: {custom_score:.2f}")


X_test shape: (2800, 10)
Predictions shape: (2800, 11)
Final RMSE: 11.59
Custom Score: 88.41


In [43]:
data.isna().sum()

Residents                     0
Apartment_Type                0
Temperature                 441
Humidity                    397
Water_Price                   0
Period_Consumption_Index      0
Income_Level                  0
Guests                        0
Amenities                     0
Appliance_Usage             415
Water_Consumption             0
dtype: int64

In [44]:
X1=data.drop('Water_Consumption',axis=1)
y1=data['Water_Consumption']
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=42)




from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

# ... (Your data loading and preprocessing code) ...

# 1. Create and train the model
model = LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# 2. Make predictions
y_pred1 = model.predict(X_test)

# 3. Evaluate the model
rmse_score = np.sqrt(mean_squared_error(y_test, y_pred1))
custom_score = max(0, 100 - rmse_score)

print(f"Final RMSE: {rmse_score:.2f}")
print(f"Custom Score: {custom_score:.2f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000493 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1007
[LightGBM] [Info] Number of data points in the train set: 11200, number of used features: 10
[LightGBM] [Info] Start training from score 164.187287
Final RMSE: 11.59
Custom Score: 88.41


In [45]:
X2=dt.drop('Water_Consumption',axis=1)
y2=dt['Water_Consumption']
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

# ... (Your data loading and preprocessing code) ...

# 1. Create and train the model
model = LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# 2. Make predictions
y_pred2 = model.predict(X_test)

# 3. Evaluate the model
rmse_score = np.sqrt(mean_squared_error(y_test, y_pred2))
custom_score = max(0, 100 - rmse_score)

print(f"Final RMSE: {rmse_score:.2f}")
print(f"Custom Score: {custom_score:.2f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000495 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1006
[LightGBM] [Info] Number of data points in the train set: 11200, number of used features: 10
[LightGBM] [Info] Start training from score 164.187287
Final RMSE: 11.54
Custom Score: 88.46


In [46]:
import joblib
joblib.dump(model, 'model.pkl')

# Load the model
loaded_model = joblib.load('model.pkl')

In [47]:
df=pd.read_csv(r'Z:\projects\smart water system\dataset\test.csv')

In [48]:
df.isna().sum()

Timestamp                      0
Residents                      0
Apartment_Type               166
Temperature                  150
Humidity                       0
Water_Price                    0
Period_Consumption_Index       0
Income_Level                 165
Guests                         0
Amenities                   2513
Appliance_Usage              177
dtype: int64

In [49]:
df.dtypes

Timestamp                    object
Residents                     int64
Apartment_Type               object
Temperature                 float64
Humidity                     object
Water_Price                 float64
Period_Consumption_Index    float64
Income_Level                 object
Guests                        int64
Amenities                    object
Appliance_Usage             float64
dtype: object

In [50]:
df = df.drop('Timestamp', axis=1)

In [51]:
df['Residents'].unique()

array([  5,   4,   3,   2,   1, -77, -99, -22, -11, -33, -88, -44, -55,
       -66], dtype=int64)

In [52]:
df['Apartment_Type'].unique()

array(['Bungalow', 'Cottage', '1BHK', '3BHK', '2BHK', 'Detached',
       'Studio', nan], dtype=object)

In [53]:
df['Apartment_Type'] = df['Apartment_Type'].fillna('Missing')

In [54]:
df['Temperature'].unique()

array([11.89, 29.22, 10.27, ..., 10.26, 13.41, 11.8 ])

In [55]:
df['Temperature'] = df['Temperature'].fillna(df['Temperature'].median())

In [56]:
df['Humidity'].unique()

array(['57.88', '61.41', '64.9', ..., '48.78', '58.68', '33.14'],
      dtype=object)

In [57]:
def convert_to_float_or_nan(value):
    """Converts a value to float if it's a numerical string,
    otherwise returns NaN."""
    try:
        # Check if the value matches a numerical pattern
        if re.fullmatch(r'\d+(\.\d+)?', value):
            return float(value)
        else:
            return np.nan  # Non-numerical, replace with NaN
    except (ValueError, TypeError):
        return np.nan  # Handle errors (e.g., if value is None)

# Apply the function to the 'Humidity' column
df['Humidity'] = df['Humidity'].apply(convert_to_float_or_nan)

In [58]:
df['Humidity']=df['Humidity'].fillna(df['Humidity'].median())

In [59]:
df['Water_Price'].unique()

array([  2.77,   2.85,   1.66,   1.48,   1.19,   1.91,   2.51,   1.64,
         2.19,   2.82,   1.85,   1.36,   1.49,   1.4 ,   1.8 ,   1.76,
         2.53,   1.45,   2.06,   2.43,   1.37,   2.46,   2.64,   1.04,
         1.6 ,   2.89,   1.78,   2.93,   2.67,   1.56, -88.  ,   1.55,
         2.84,   2.44,   2.1 ,   2.99,   1.32,   2.71,   1.21,   1.41,
         1.53,   1.57,   1.09,   1.59,   2.22,   1.29,   2.35,   1.16,
         1.39,   1.31,   2.17,   1.25,   1.68,   1.46,   1.38,   2.37,
         2.15,   1.47,   1.62,   2.39,   1.22,   1.71,   1.79,   1.61,
         2.72,   1.72,   2.41,   1.3 ,   1.77,   1.99,   1.5 ,   1.05,
         1.06,   1.2 ,   1.18,   1.11,   2.49,   2.47,   1.75,   1.58,
         1.51,   1.63,   1.86,   1.35,   2.18,   1.65,   1.88,   1.82,
         1.1 ,   1.42,   2.32,   2.73,   1.24,   1.7 ,   2.28,   1.9 ,
         2.31,   2.26,   1.73,   1.26,   2.86,   1.43,   1.15,   2.75,
         1.28,   1.07,   1.08, -44.  ,   2.95,   1.12,   1.34,   2.02,
      

In [60]:
df['Period_Consumption_Index'].unique()

array([ 1.48      ,  1.3       ,  0.97      ,  1.31      ,  1.14      ,
        1.06      ,  1.13      ,  1.11      ,  1.49      ,  1.4       ,
        1.12      ,  1.1       ,  1.        ,  0.83      ,  1.5       ,
        1.43      ,  1.04      ,  0.82      ,  1.47      ,  1.39      ,
        0.9       ,  1.09      ,  1.42      ,  1.38      ,  1.03      ,
        0.91      ,  0.98      ,  0.88      ,  0.96      ,  1.35      ,
        1.44      ,  1.23      ,  0.94      ,  1.26      ,  1.08      ,
        1.19      ,  1.37      ,  1.25      ,  1.41      ,  2.16861286,
        1.2       ,  0.8       ,  0.85      ,  0.86      ,  0.71003935,
        1.15      ,  0.89      ,  1.27      ,  1.24      ,  1.36      ,
        1.45      ,  1.02      ,  1.18      ,  0.87      ,  1.28      ,
        1.48785133,  1.22      ,  1.16      ,  0.81      ,  0.93      ,
        0.92      ,  1.17      ,  1.34      ,  1.05      ,  2.08592223,
        1.01      ,  0.95      ,  1.07      ,  1.32      ,  1.21

In [61]:
df['Income_Level'].unique()

array(['Upper Middle', 'Middle', 'Low', 'Rich', '!vg],', nan, '|O6&W',
       "'#rm&", '#q7B0', 'Bo%vV', '1S\\09', 'hvNL}', 'G0;U$', 'L8"Ju',
       '0OC_i', '${n)t', 'L?-2l', 'fT(ao', '?n\\3L', 'oYo=v', '>m3Q|',
       'vXO|U', 'H7EFe', '{{\\3I', '-USqL', ')DK~A', 'P.;bp', 'K7d"U',
       'uVDI-', 'e4~3"', 'doJ?D', '(i8m/', 't5of{', '|b!Q$', 'f]Q*~',
       'oNI_s', '?33Od', "^'Hg1", 'kIt`=', '3&K8&', '2rp4.', '7S8e3',
       '~$;fC', 'uc~vk', 'e3eoi', 'I{2i4', 'PZvuy', '5P>_Q', '0W.!W',
       'r"2CI', 'A%=OJ', ":kU^'", '09l]%', '`gV~e', 'zMf60', '*=SUH',
       '?_^nL', 'hZ`Tf', '=JlF/', 'dj#U+', 'LlUBT', 'S!|`r', ']|G/A',
       'uu:lz', 'An"Rk', '+9-mY', 'oswQ|', 'W2Zs2', ")~C'l", '<ZP2"',
       '<pe]T', 'aM{L?', 'H~)`Y', '_l=u<', 'c4XdG', 'J@opf', 'qyuu#',
       'x4~#E', 'T!<9<', '`wY0/', 'OvU0[', 'T*@P#', 'C>7PT', 'N`%/L',
       'X8Vz&', 'T00<7', 'A0>C4', '_N~m/', '}TJ.1', 'qv-X(', 'SHZ:R',
       'ZOM5q', 'L~+QP', 'r&;v8', '^5Pee', "l'3|Q", 'Vp&%_', "A?'t(",
       'fC\\Xk',

In [62]:
valid_categories = ['Low', 'Middle', 'Upper Middle', 'Rich']

# 2. Replace unusual values with 'Unknown'
df['Income_Level'] = df['Income_Level'].apply(lambda x: x if x in valid_categories else 'Unknown')


In [63]:
df['Income_Level'] = df['Income_Level'].fillna('Missing')

In [64]:
df['Guests'].unique()

array([ 1,  0, -1,  2,  3], dtype=int64)

In [65]:
df['Amenities'].unique()

array(['Jacuzzi', 'Garden', nan, 'Swimming Pool', 'Fountain'],
      dtype=object)

In [66]:
df['Amenities'] = df['Amenities'].fillna('Missing')

In [67]:
df['Appliance_Usage'].unique()

array([ 0., nan,  1.])

In [68]:
df['Appliance_Usage'] = df['Appliance_Usage'].fillna(df['Appliance_Usage'].median())

In [69]:
df.isna().sum()

Residents                   0
Apartment_Type              0
Temperature                 0
Humidity                    0
Water_Price                 0
Period_Consumption_Index    0
Income_Level                0
Guests                      0
Amenities                   0
Appliance_Usage             0
dtype: int64

In [70]:
df.dtypes

Residents                     int64
Apartment_Type               object
Temperature                 float64
Humidity                    float64
Water_Price                 float64
Period_Consumption_Index    float64
Income_Level                 object
Guests                        int64
Amenities                    object
Appliance_Usage             float64
dtype: object

In [71]:
encoder = LabelEncoder()

# Select the categorical columns you want to encode
categorical_columns = ['Apartment_Type', 'Amenities','Income_Level']  # Replace with your column names

# Loop through the columns and apply label encoding
for column in categorical_columns:
    df[column] = encoder.fit_transform(df[column])

In [72]:
df.dtypes

Residents                     int64
Apartment_Type                int32
Temperature                 float64
Humidity                    float64
Water_Price                 float64
Period_Consumption_Index    float64
Income_Level                  int32
Guests                        int64
Amenities                     int32
Appliance_Usage             float64
dtype: object

In [73]:
df.columns

Index(['Residents', 'Apartment_Type', 'Temperature', 'Humidity', 'Water_Price',
       'Period_Consumption_Index', 'Income_Level', 'Guests', 'Amenities',
       'Appliance_Usage'],
      dtype='object')

In [74]:
y_pred3 = model.predict(df)

# Convert to DataFrame
predictions_df = pd.DataFrame({'Predicted': y_pred3})

# Save to CSV
predictions_df.to_csv('predictions.csv', index=False)

print("Predictions saved to 'predictions.csv'")


Predictions saved to 'predictions.csv'


In [76]:
final=pd.read_csv(r'Z:\projects\smart water system\dataset\test.csv')
pred=pd.read_csv(r'Z:\projects\smart water system\predictions.csv')

In [77]:
pred

,Predicted
0,300.570492
1,188.863666
2,77.739536
3,122.470032
4,124.849289
...,...
5995,236.394150
5996,83.689142
5997,263.052839
5998,183.645250


In [78]:
final.columns

Index(['Timestamp', 'Residents', 'Apartment_Type', 'Temperature', 'Humidity',
       'Water_Price', 'Period_Consumption_Index', 'Income_Level', 'Guests',
       'Amenities', 'Appliance_Usage'],
      dtype='object')

In [79]:
final=final.drop(columns=['Residents', 'Apartment_Type', 'Temperature', 'Humidity',
       'Water_Price', 'Period_Consumption_Index', 'Income_Level', 'Guests',
       'Amenities', 'Appliance_Usage'],axis=1)

In [80]:
final

,Timestamp
0,11/10/2014 16
1,12/10/2014 00
2,12/10/2014 08
3,12/10/2014 16
4,13/10/2014 00
...,...
5995,01/04/2020 00
5996,01/04/2020 08
5997,01/04/2020 16
5998,02/04/2020 00


In [81]:
merged_df = pd.concat([final, pred], axis=1)

In [82]:
merged_df.to_csv('final.csv',index=False)